## Import data from Spreadsheet Template, and prepare it for a PyCap run

### Use the spreadsheet template found here, do not make alterations to column names in the spreadsheet or this script may not work

##### Libraries you will need to run this script

In [ ]:
import yaml
import pandas as pd
import numpy as np


### Path to your .xlsx template spreadsheet
##### Note the use of double backslashes in the path name
##### Also provide a name for the PyCap run

In [ ]:
pycap_inputs_excel = "./test_run.xlsx"
pycap_run_name = 'TestExample'

In [ ]:
raw_global = pd.read_excel(pycap_inputs_excel, sheet_name = 'Global_Inputs')
raw_hcw = pd.read_excel(pycap_inputs_excel, sheet_name = 'HCW_Inputs')
raw_dd = pd.read_excel(pycap_inputs_excel, sheet_name = 'Drawdown_Inputs')
raw_depl = pd.read_excel(pycap_inputs_excel, sheet_name ='Depletion_Inputs')

#### Data wrangling to get excel readable in pandas

In [ ]:
raw_hcw['HCW'] = raw_hcw['HCW'].astype(str)
raw_dd['HCW'] = raw_dd['HCW'].astype(str)
raw_depl['HCW'] = raw_depl['HCW'].astype(str)
raw_dd['pycap_resource_name'] = raw_dd['Resource_Name']
raw_depl['pycap_resource_name'] = raw_depl['Resource_Name']

for i in range(len(raw_dd)):
    raw_dd['Resource_Name'][i] = raw_dd['Resource_Name'][i].replace(" ","")
    raw_dd['pycap_resource_name'][i] = ":".join([raw_dd['Resource_Name'][i], raw_dd['HCW'][i]])

for i in range(len(raw_depl)):
    raw_depl['Resource_Name'][i] = raw_depl['Resource_Name'][i].replace(" ","")
    raw_depl['pycap_resource_name'][i] = ":".join([raw_depl['Resource_Name'][i], raw_depl['HCW'][i]])


#### Create nested dictionaries of inputs for YML

##### Project data, with default data for T, S, t, etc

In [ ]:
project_dict = dict()
project_dict['project_properties'] = {'name':pycap_run_name,
                                     'T':float(raw_global['Transmissivity_ft2d'][0]),
                                     'S':float(raw_global['Storage_Coeff'][0]),
                                      'default_dd_days':float(raw_global['Default_dd_days'][0]),
                                      'default_depletion_years':float(raw_global['Default_depletion_years']),
                                      'default_pumping_days':float(raw_global['Default_pumping_days'])
                                     }

##### Well data, with apportionment, and name of location impacts will be assessed

In [ ]:
well_dict=dict()
stream_dict=dict()

for j in range(len(raw_depl['HCW'])):
    stream_dict[j] = {'HCW':raw_depl['HCW'][j],
                      'stream_apportionment':{#f'stream_apportionment{j}':{
        'name':raw_depl['pycap_resource_name'][j],
        'apportionment':float(raw_depl['Fraction_Intercept'][j])
    }}
    
for i in range(len(raw_hcw)):
        well_dict[i] = {'name': raw_hcw['HCW'][i],
                   'status': raw_hcw['Well_Status'][i].lower(),
                    'loc':{
                        'x':float(raw_hcw['Well_Long'][i]),
                        'y':float(raw_hcw['Well_Lat'][i])
                        },
                    'Q':float(raw_hcw['Q_gpm'][i]),
                    'pumping_days':int(raw_hcw['Pumping_Days'][i]),
                       }
stream_dict_df = pd.DataFrame.from_dict(stream_dict,orient='index')
stream_dict_df = stream_dict_df.rename('stream_apportionment{}'.format)

for i in range(len(well_dict)):
    k_old = i
    k_new = str(well_dict[i]['name'])
    well_dict[k_new] = well_dict.pop(k_old)

for j in well_dict:
    well_dict[j].update(stream_dict_df.loc[stream_dict_df['HCW']==j]['stream_apportionment'])

for j in well_dict:
    well_dict[j].update({'stream_response':(list(raw_depl.loc[raw_depl['HCW']==j]['pycap_resource_name']))})

for j in well_dict:
    well_dict[j].update({'dd_response': (list(raw_dd.loc[raw_dd['HCW']==j]['Resource_Name']))})

for key in list(well_dict):
    k_new = "well_{}".format(key)
    well_dict[k_new] = well_dict.pop(key)


##### Stream Response Locations

In [ ]:
streamresp_dict = dict()
for i in range(len(raw_depl)):
    streamresp_dict['stream_response{}'.format(i)]={
        'name': raw_depl['pycap_resource_name'][i],
        'loc': {
            'x': float(raw_depl['Resource_Long'][i]),
            'y': float(raw_depl['Resource_Lat'][i])
            }
        }

##### Drawdown Response Locations

In [ ]:
raw_dd_unique = raw_dd.drop_duplicates(subset=['Resource_Name']).reset_index()
ddresp_dict = dict()
for i in range(len(raw_dd_unique)):
    ddresp_dict['dd_response{}'.format(i)]={
        'name':raw_dd_unique['Resource_Name'][i],
        'loc':{
            'x':float(raw_dd_unique['Resource_Long'][i]),
            'y':float(raw_dd_unique['Resource_Lat'][i])
        }
    }


#### Combine all the above dictionaries and save as a yml

In [ ]:
combdict={**project_dict,**well_dict, **ddresp_dict, **streamresp_dict}
yml_name = (pycap_run_name+'.yml')

with open(yml_name,'w') as file:
    documents = yaml.dump(combdict, file, default_flow_style = False, sort_keys= False)